In [1]:
import pandas as pd
import os
from tqdm import tqdm
import polars as pl 




In [2]:
def read_excel_header(file_path, skip=0):
    """Attempt to read just the header of an Excel file with varying skip rows."""
    try:
        col_list = pd.read_excel(file_path, nrows=0, skiprows=skip).columns
        return col_list
    except Exception as e:
        print(f"Error reading header from {file_path}: {e}")
        return []

def find_unique_schemas(directory):
    """Finds and returns unique schemas from Excel files in a given directory."""
    schemas = {}
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            try:
                columns = read_excel_header(file_path)
                num_columns = len(columns)
                skip_r=1
                while num_columns <3:
                    columns = read_excel_header(file_path, skip=skip_r)
                    num_columns = len(columns)
                    skip_r+=1
                schema_key = tuple(columns)  # Use tuple of columns as a unique schema key
            except Exception as e:
                print(f"Failed to open file {filename}: {e}")

            if schema_key not in schemas:
                schemas[schema_key] = []
            schemas[schema_key].append(filename)
    return schemas
    

In [3]:
# Usage example
directory = 'downloaded_files'
unique_schemas = find_unique_schemas(directory)

# Printing unique schemas and the files that match them
#for schema, files in unique_schemas.items():
#    print(f"Schema: {schema} \nFiles: {files}\n")

100%|██████████| 68/68 [00:07<00:00,  9.27it/s]


In [4]:
# Sort elements within each tuple and deduplicate
sorted_unique_schemas = set(tuple(sorted(schema)) for schema in unique_schemas.keys())


In [5]:
schema1 = list(unique_schemas.values())[0]
schema2 = list(unique_schemas.values())[1]
schema3 = list(unique_schemas.values())[2]
schema4 = list(unique_schemas.values())[3]
schema5 = list(unique_schemas.values())[4]
schema6 = list(unique_schemas.values())[5]
schema7 = list(unique_schemas.values())[6]
schema8 = list(unique_schemas.values())[7]
schema9 = list(unique_schemas.values())[8]
schema10 = list(unique_schemas.values())[9]
schema11 = list(unique_schemas.values())[10]
schema12 = list(unique_schemas.values())[11]
schema13 = list(unique_schemas.values())[12]
schema14 = list(unique_schemas.values())[13]
schema15 = list(unique_schemas.values())[14]

In [6]:
# Schema definition moved outside of the loop
final_schema_columns = ['Fecha', 'Año', 'Mes', 'Sigla Empresa', 'Nombre Empresa',
                        'Origen', 'Destino', 'Apto_Origen', 'Apto_Destino',
                        'Pasajeros', 'Trafico', 'TipoVuelo', 'Ciudad Origen', 
                        'Ciudad Destino', 'Pais Origen', 'Pais Destino']

In [103]:
# Processing loop
df_list = []
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        skip_r = 0
        columns = read_excel_header(file_path, skip=skip_r)
        while len(columns) < 3 and skip_r < 10:
            columns = read_excel_header(file_path, skip=skip_r)
            skip_r += 1

        try:
            tmp = pd.read_excel(file_path, skiprows=skip_r, dtype="str")
            if (filename in schema1) | (filename in schema5)  | (filename in schema6) | (filename in schema11):
                tmp["Año"] = tmp['Fecha'].str.slice(start=0, stop=4)
                tmp["Mes"] = tmp['Fecha'].str.slice(start=5, stop=7)
                tmp = tmp[final_schema_columns]
                pl_tmp = pl.from_pandas(tmp)
                df_list.append(pl_tmp)
            elif (filename in schema2) | (filename in schema3) | (filename in schema4) | (filename in schema8) | (filename in schema12) :
                tmp = tmp.rename(columns={"Nombre":'Nombre Empresa',"Número de Mes":"Mes","MES":"Mes",
                                          "Nombre.1":'Apto_Origen',"Nombre.2":'Apto_Destino',
                                          "Tráfico (N/I)":'Trafico',"Tipo Vuelo":'TipoVuelo',
                                          "Apto Origen":'Apto_Origen',"Apto Destino":'Apto_Destino',
                                          "Fecha Visual":"Fecha", "Tráfico":"Trafico", "AÑO":"Año"}) 
                tmp = tmp[final_schema_columns]
                pl_tmp = pl.from_pandas(tmp)
                df_list.append(pl_tmp)
            elif filename in schema7:
                tmp = pd.read_excel(file_path, skiprows=1, dtype="str", sheet_name=1)
                tmp = tmp.rename(columns={"Nombre":'Nombre Empresa',"Número de Mes":"Mes","MES":"Mes",
                                          "Nombre.1":'Apto_Origen',"Nombre.2":'Apto_Destino',
                                          "Tráfico (N/I)":'Trafico',"Tipo Vuelo":'TipoVuelo',
                                          "Apto Origen":'Apto_Origen',"Apto Destino":'Apto_Destino',
                                          "Fecha Visual":"Fecha", "Tráfico":"Trafico", "AÑO":"Año"}) 
                tmp = tmp[final_schema_columns]
                pl_tmp = pl.from_pandas(tmp)
                df_list.append(pl_tmp)
            else:
                continue
        except Exception as e:
            print(f"Failed to process file {filename}: {e}")

# Concatenate all dataframes
df = pl.concat(df_list)

  0%|          | 0/68 [00:00<?, ?it/s]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 21%|██        | 14/68 [00:25<03:57,  4.39s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 28%|██▊       | 19/68 [00:30<01:12,  1.48s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 41%|████      | 28/68 [00:43<00:41,  1.04s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header

Failed to process file Origen-Destino Mes 2014.xlsx: "['Fecha'] not in index"


100%|██████████| 68/68 [02:05<00:00,  1.85s/it]

Failed to process file Origen-Destino Mes 2015.xlsx: "['Fecha'] not in index"


In [7]:
import os
import pandas as pd
import polars as pl
from tqdm import tqdm

# Assuming the function definitions for read_excel_header and find_unique_schemas are unchanged

# Usage example
directory = 'downloaded_files'
unique_schemas = find_unique_schemas(directory)

# Mapping filenames to their schema index for better handling
filename_to_schema_index = {filename: index for index, schemas in enumerate(unique_schemas.values()) for filename in schemas}

# Schema mappings for renaming and adjustments
schema_mappings = {
    # Group similar handling schemas together
    'group1': [0, 4, 5, 10],
    'group2': [1, 2, 3, 7, 11],
    'group3': [6]
}

# Processing loop
df_list = []
for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        skip_r = 0
        columns = read_excel_header(file_path, skip=skip_r)
        while len(columns) < 3 and skip_r < 10:
            skip_r += 1
            columns = read_excel_header(file_path, skip=skip_r)

        schema_index = filename_to_schema_index.get(filename)
        try:
            group_key = next(key for key, indexes in schema_mappings.items() if schema_index in indexes)
            tmp = pd.read_excel(file_path, skiprows=skip_r, dtype="str")
            
            if group_key == 'group1':
                tmp["Año"] = tmp['Fecha'].str.slice(start=0, stop=4)
                tmp["Mes"] = tmp['Fecha'].str.slice(start=5, stop=7)
            elif group_key == 'group2':
                tmp = tmp.rename(columns={"Nombre": 'Nombre Empresa', "Número de Mes": "Mes", "MES": "Mes",
                                          "Nombre.1": 'Apto_Origen', "Nombre.2": 'Apto_Destino',
                                          "Tráfico (N/I)": 'Trafico', "Tipo Vuelo": 'TipoVuelo',
                                          "Apto Origen": 'Apto_Origen', "Apto Destino": 'Apto_Destino',
                                          "Fecha Visual": "Fecha", "Tráfico": "Trafico", "AÑO": "Año"})
            elif group_key == 'group3':
                tmp = pd.read_excel(file_path, skiprows=1, dtype="str", sheet_name="DATOS")
            
            tmp = tmp[final_schema_columns]
            pl_tmp = pl.from_pandas(tmp)
            df_list.append(pl_tmp)

        except Exception as e:
            print(f"Failed to process file {filename}: {e}")

# Concatenate all dataframes
df = pl.concat(df_list)


  0%|          | 0/68 [00:00<?, ?it/s]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 21%|██        | 14/68 [00:26<04:07,  4.58s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 28%|██▊       | 19/68 [00:31<01:20,  1.64s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 34%|███▍      | 23/68 [00:40<01:21,  1.82s/it]

Failed to process file Base de Datos Origen - Destino Febrero 2023.xlsx: Worksheet index 1 is invalid, 1 worksheets found


 38%|███▊      | 26/68 [00:42<00:45,  1.09s/it]

Failed to process file Base de Datos Origen - Destino Julio 2021.xlsx: 


 41%|████      | 28/68 [00:44<00:41,  1.04s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 47%|████▋     | 32/68 [00:48<00:37,  1.05s/it]d:\Proyectos\air.traffic.trends-CO\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
 59%|█████▉    | 40/68 [00:58<00:37,  1.35s/it]

...

Failed to process file Bases de Datos Origen-Destino Febrero 2020.xlsx: 
Failed to process file Bases de Datos Origen-Destino Junio 2020.xlsx: 
Failed to process file Origen - Destino Mes 2016.xlsx: 


 99%|█████████▊| 67/68 [01:24<00:00,  2.67it/s]

Failed to process file Origen-Destino Mes 2014.xlsx: 


100%|██████████| 68/68 [01:25<00:00,  1.25s/it]

Failed to process file Origen-Destino Mes 2015.xlsx: 


In [8]:
df

Fecha,Año,Mes,Sigla Empresa,Nombre Empresa,Origen,Destino,Apto_Origen,Apto_Destino,Pasajeros,Trafico,TipoVuelo,Ciudad Origen,Ciudad Destino,Pais Origen,Pais Destino
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2019-04-01 00:00:00""","""2019""","""04""","""AAL""","""AMERICAN""","""ABI""","""BOG""","""MUNICIPAL""","""BOGOTA - ELDORADO""","""1""","""I""","""R""","""TEXAS""","""BOGOTA""","""ESTADOS UNIDOS""","""COLOMBIA"""
"""2019-04-01 00:00:00""","""2019""","""04""","""AAL""","""AMERICAN""","""ABQ""","""BOG""","""ALBUQUERQUE INTL SUNPORT""","""BOGOTA - ELDORADO""","""37""","""I""","""R""","""NUEVO MEXICO""","""BOGOTA""","""ESTADOS UNIDOS""","""COLOMBIA"""
"""2019-04-01 00:00:00""","""2019""","""04""","""AAL""","""AMERICAN""","""ABZ""","""CLO""","""CYDE""","""CALI - ALFONSO BONILLA ARAGON""","""1""","""I""","""R""","""ABENDEEN""","""CALI""","""INGLATERRA""","""COLOMBIA"""
"""2019-04-01 00:00:00""","""2019""","""04""","""AAL""","""AMERICAN""","""AMS""","""CTG""","""SCHIPHOL""","""CARTAGENA - RAFAEL NUQEZ""","""2""","""I""","""R""","""AMSTERDAM""","""CARTAGENA""","""HOLANDA""","""COLOMBIA"""
"""2019-04-01 00:00:00""","""2019""","""04""","""AAL""","""AMERICAN""","""ANU""","""MDE""","""ANTIGUA LEEWARDS V.C.BIRD,""","""RIONEGRO - JOSE M. CORDOVA""","""1""","""I""","""R""","""ST. JOHN'S""","""RIONEGRO - ANTIOQUIA""","""ANTIGUA AND BARBUDA""","""COLOMBIA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2019-May""","""2019""","""5""","""WGN""","""WESTERN GLOBAL""","""BOG""","""MIA""","""BOGOTA - ELDORADO""","""MIAMI INTL""","""0""","""I""","""C""","""BOGOTA""","""MIAMI""","""COLOMBIA""","""ESTADOS UNIDOS"""
"""2019-May""","""2019""","""5""","""WGN""","""WESTERN GLOBAL""","""DFW""","""BOG""","""DALLAS""","""BOGOTA - ELDORADO""","""0""","""I""","""C""","""DALAS""","""BOGOTA""","""ESTADOS UNIDOS""","""COLOMBIA"""
"""2019-May""","""2019""","""5""","""WGN""","""WESTERN GLOBAL""","""MDE""","""MIA""","""RIONEGRO - JOSE M. CORDOVA""","""MIAMI INTL""","""0""","""I""","""C""","""RIONEGRO - ANTIOQUIA""","""MIAMI""","""COLOMBIA""","""ESTADOS UNIDOS"""


In [ ]:
df = pl.DataFrame(schema=
             {'Fecha':pl.Utf8,"Año":pl.Utf8,"Mes":pl.Utf8,'Sigla Empresa':pl.Utf8,'Nombre Empresa':pl.Utf8,
              'Origen':pl.Utf8, 'Destino':pl.Utf8, 'Apto_Origen':pl.Utf8,'Apto_Destino':pl.Utf8,
              'Pasajeros':pl.Utf8, 'Trafico':pl.Utf8,'TipoVuelo':pl.Utf8, 'Ciudad Origen':pl.Utf8, 'Ciudad Destino':pl.Utf8,
              'Pais Origen':pl.Utf8, 'Pais Destino':pl.Utf8})

In [ ]:
a = list(unique_schemas.values())[0]
filename  = a[0]
file_path = os.path.join(directory, filename)
columns = read_excel_header(file_path)
num_columns = len(columns)
skip_r=1

num_columns

while num_columns <3:
    columns = read_excel_header(file_path, skip=skip_r)
    num_columns = len(columns)
    skip_r+=1
columns

pd.read_excel(file_path, skiprows=skip_r-1)

In [ ]:
df = pl.DataFrame(schema=
             {'Fecha':pl.Utf8,"Año":pl.Utf8,"Mes":pl.Utf8,'Sigla Empresa':pl.Utf8,'Nombre Empresa':pl.Utf8,
              'Origen':pl.Utf8, 'Destino':pl.Utf8, 'Apto_Origen':pl.Utf8,'Apto_Destino':pl.Utf8,
              'Pasajeros':pl.Utf8, 'Trafico':pl.Utf8,'TipoVuelo':pl.Utf8, 'Ciudad Origen':pl.Utf8, 'Ciudad Destino':pl.Utf8,
              'Pais Origen':pl.Utf8, 'Pais Destino':pl.Utf8})

for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            try:
                columns = read_excel_header(file_path)
                num_columns = len(columns)
                skip_r=1
                while num_columns <3:
                    columns = read_excel_header(file_path, skip=skip_r)
                    num_columns = len(columns)
                    skip_r+=1
                tmp = pd.read_excel(file_path, skiprows=skip_r-1, dtype="str")
                if filename in list(unique_schemas.values())[0]:
                     tmp["Año"] = ""
                     tmp["Mes"] = ""
                     tmp = tmp[['Fecha',"Año","Mes",'Sigla Empresa','Nombre Empresa',
                                  'Origen', 'Destino', 'Apto_Origen','Apto_Destino',
                                'Pasajeros', 'Trafico','TipoVuelo', 'Ciudad Origen', 'Ciudad Destino',
                                'Pais Origen', 'Pais Destino']]
                     tmp = pl.DataFrame(tmp)
                     tmp = tmp.with_columns(pl.col("Fecha").str.to_datetime().dt.year().alias("Año").cast(pl.Utf8),
                                     pl.col("Fecha").str.to_datetime().dt.month().alias("Mes").cast(pl.Utf8))
                     df = pl.concat([df,tmp])
                else:
                     continue            
            except Exception as e:
                print(f"Failed to open file {filename}: {e}")

In [ ]:
import os
file = list(unique_schemas.values())[0][1]
file_path = os.path.join(directory, file)

In [ ]:
tmp

In [ ]:
tmp = pd.read_excel(file_path, skiprows=skip_r, dtype="str")
tmp["Año"] = ""
tmp["Mes"] = ""
tmp = tmp[['Fecha',"Año","Mes",'Sigla Empresa','Nombre Empresa',
                                  'Origen', 'Destino', 'Apto_Origen','Apto_Destino',
                                'Pasajeros', 'Trafico','TipoVuelo', 'Ciudad Origen', 'Ciudad Destino',
                                'Pais Origen', 'Pais Destino']]
tmp = pl.DataFrame(tmp)

tmp

In [ ]:
tmp.with_columns(pl.col("Fecha").str.to_datetime().dt.year().alias("Año").cast(pl.Utf8),
                                     pl.col("Fecha").str.to_datetime().dt.month().alias("Mes").cast(pl.Utf8))

In [ ]:
dict_schemas["schema 1"] == dict_schemas["schema 1"]

In [ ]:
schemas = list(unique_schemas.keys())
len(schemas[0])

In [ ]:
#print(pd.Series(schemas[0]).sort_values().reset_index(drop=True))


In [ ]:
print(pd.Series(schemas[1]).sort_values().reset_index(drop=True))

In [ ]:
#Harmonizar
CargaKg Carga (Kg) | CorreoKg Correo (Kg) | Nombre Empresa Nombre | Tipo Vuelo TipoVuelo |Trafico Tráfico (N/I)
#agreegar a todos
Año  Número de Mes

In [ ]:
pd.Series(schemas[2]).sort_values()

In [ ]:
len(schemas[2])